In [ ]:
import copy
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
cardboard = []
for i in range(1,404):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/cardboard/cardboard"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(224,224))
    X = np.array(temp)
    cardboard.append(X)
cardboard = np.array(cardboard)
cardboard = np.take(cardboard,np.random.permutation(cardboard.shape[0]),axis=0)
print(cardboard.shape)

In [ ]:
glass = []
for i in range(1,502):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/glass/glass"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(224,224))
    X = np.array(temp)
    glass.append(X)
glass = np.array(glass)
glass = np.take(glass,np.random.permutation(glass.shape[0]),axis=0)
print(glass.shape)

In [ ]:
metal = []
for i in range(1,411):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/metal/metal"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(224,224))
    X = np.array(temp)
    metal.append(X)
metal = np.array(metal)
metal = np.take(metal,np.random.permutation(metal.shape[0]),axis=0)
print(metal.shape)

In [ ]:
paper = []
for i in range(1,595):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/paper/paper"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(224,224))
    X = np.array(temp)
    paper.append(X)
paper = np.array(paper)
paper = np.take(paper,np.random.permutation(paper.shape[0]),axis=0)
print(paper.shape)

In [ ]:
plastic = []
for i in range(1,483):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/plastic/plastic"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(224,224))
    X = np.array(temp)
    plastic.append(X)
plastic = np.array(plastic)
plastic = np.take(plastic,np.random.permutation(plastic.shape[0]),axis=0)
print(plastic.shape)

In [ ]:
trash = []
for i in range(1,138):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/trash/trash"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(224,224))
    X = np.array(temp)
    trash.append(X)
trash = np.array(trash)
trash = np.take(trash,np.random.permutation(trash.shape[0]),axis=0)
print(trash.shape)

In [ ]:
train_X = np.vstack((cardboard[:302,:,:,:],glass[:376,:,:,:],metal[:307,:,:,:],
                    paper[:445,:,:,:],plastic[:361,:,:,:],trash[:103,:,:,:]))
test_X = np.vstack((cardboard[302:,:,:,:],glass[376:,:,:,:],metal[307:,:,:,:],
                   paper[445:,:,:,:],plastic[361:,:,:,:],trash[103:,:,:,:]))
train_Y = np.zeros((6,train_X.shape[0]))
test_Y = np.zeros((6,test_X.shape[0]))
for i in range(train_X.shape[0]):
    if(i<302):
        train_Y[0][i] = 1
    elif(i<678):
        train_Y[1][i] = 1
    elif(i<985):
        train_Y[2][i] = 1
    elif(i<1430):
        train_Y[3][i] = 1
    elif(i<1791):
        train_Y[4][i] = 1
    else:
        train_Y[5][i] = 1
for i in range(test_X.shape[0]):
    if(i<101):
        test_Y[0][i] = 1
    elif(i<226):
        test_Y[1][i] = 1
    elif(i<329):
        test_Y[2][i] = 1
    elif(i<478):
        test_Y[3][i] = 1
    elif(i<599):
        test_Y[4][i] = 1
    else:
        test_Y[5][i] = 1

In [ ]:
np.random.seed(1013)
train_perm = np.random.permutation(train_X.shape[0])
train_X = np.take(train_X,train_perm,axis=0)
print(train_X.shape)
train_Y = np.take(train_Y,train_perm,axis=1)
print(train_Y.shape)
np.random.seed(901)
test_perm = np.random.permutation(test_X.shape[0])
test_X = np.take(test_X,test_perm,axis=0)
print(test_X.shape)
test_Y = np.take(test_Y,test_perm,axis=1)
print(test_Y.shape)

In [ ]:
def VGG11(input_shape = (224, 224, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = tf.keras.layers.Input(input_shape)
    
    # Stage 1
    X = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu")(X_input)
    #X = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(X)
    
    X = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(X)
    #X = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(X)
    
    X = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(X)
    X = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(X)
    #X = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(X)
    
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(X)
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(X)
    #X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(X)
    
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(X)
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(X)
    #X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(X)

    X = tf.keras.layers.Flatten()(X)
    #X = tf.keras.layers.Dropout(0.5)(X)
    X = tf.keras.layers.Dense(units=4096,activation="relu")(X)
    #X = tf.keras.layers.Dropout(0.5)(X)
    X = tf.keras.layers.Dense(units=4096,activation="relu")(X)
    
    X = tf.keras.layers.Dense(classes, activation="softmax")(X)
    
    # Create model
    model = tf.keras.Model(inputs = X_input, outputs = X, name='VGG16')

    return model

In [ ]:
model = VGG11()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_X, train_Y.T, epochs = 40, batch_size = 32)

In [ ]:
plt.plot(history.history["accuracy"])
plt.show()

In [ ]:
preds = model.evaluate(test_X, test_Y.T)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
model.summary()